In [ ]:
#read the README file for initial conda setup in the console and after run teh following line
pip install .

Processing /Users/jelle/Library/CloudStorage/OneDrive-Personal/GitHub/Digital-twin
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'Digital-twin' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'Digital-twin'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [2 lines of output]
      running bdist_wheel
      error: error in 'egg_base' option: 'src' does not exist or is not a directory
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for Digital-t

In [7]:
import fiona
import geopandas as gpd

In [18]:
# Pad naar je bestand
# Download bestuurlijke grenzen van PDOK.nl
path = r"data\bestuurlijkegrenzen.gpkg".replace("\\", "/")

In [19]:
# Bekijk alle lagen
layers = fiona.listlayers(path)
print(layers)  # ['gemeenten', 'provincies', 'landsgrens']

['gemeenten', 'provincies', 'landsgrens']


In [20]:
# import gemeentegebied alkmaar
gdf = gpd.read_file(path, layer="gemeenten")
alkmaar = gdf[gdf["gemeentenaam"] == "Alkmaar"]

# Controleer CRS
print(alkmaar.crs)

EPSG:28992


In [21]:
# Zet om naar WGS84 (lon/lat)
alkmaar_WGS84 = alkmaar.to_crs(epsg=4326)

# Kies de eerste polygon als deze een MultiPolygon is
if alkmaar_WGS84.geometry.iloc[0].geom_type == "MultiPolygon":
    alkmaar_WGS84.geometry = alkmaar_WGS84.geometry.apply(
        lambda geom: geom.geoms[0] if geom.geom_type == "MultiPolygon" else geom
    )

# Opslaan als nieuwe, enkelvoudige polygon
alkmaar_WGS84.to_file("data/alkmaar_single_polygon.geojson", driver="GeoJSON")